In [3]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import random

from auxiliaries import *

random.seed(34)

In [4]:
# types of routes
types_helsinki = {
    0: "tram",
    1: "subway",
    2: "rail",
    3: "bus",
    4: "ferry",
    5: "walking",
}

In [7]:
# STEP 1
# read in public transport network of helsinki and walking network
helsinki, helsinki_nodes = read_in_network("helsinki", "combined")
helsinki_walking, nodes_t = read_in_network("helsinki", "walk")

# adapt helsinki_walking dataframe to match helsinki dataframe
helsinki_walking = helsinki_walking.rename(columns={"d_walk": "duration_avg"})
# filter out edges that are not in helsinki_nodes
helsinki_walking = helsinki_walking[helsinki_walking["from_stop_I"].isin(helsinki_nodes)]
helsinki_walking = helsinki_walking[helsinki_walking["to_stop_I"].isin(helsinki_nodes)]

# combine two lines above into 1
helsinki_walking = helsinki_walking[
    helsinki_walking["from_stop_I"].isin(helsinki_nodes)
    & helsinki_walking["to_stop_I"].isin(helsinki_nodes)
]

helsinki_walking["duration"] = helsinki_walking["duration_avg"].apply(distance_to_duration)
helsinki_walking["n_vehicles"] = 0
helsinki_walking["route_I_counts"] = 0
helsinki_walking["route_type"] = 5

# add helsinki_walking to helsinki
helsinki_full = pd.concat([helsinki, helsinki_walking], ignore_index=True)
helsinki_full

from_stop_I  to_stop_I    d  duration_avg  n_vehicles   
0               1          2  145     60.000000          27  \
1               2         11  155     42.222222          27   
2               3          9  238     60.000000          27   
3               3          5  212     60.000000          50   
4               4          8  246     77.647059          51   
...           ...        ...  ...           ...         ...   
9067         7450       7407  593     60.000000           9   
9068         7522       7440  376      5.000000          12   
9069         7523       7525  388      0.000000          13   
9070         7524       7522  432     55.000000          12   
9071         7525       7398  264     50.769231          13   

               route_I_counts  route_type  duration  
0                       21:27           3       NaN  
1                       21:27           3       NaN  
2                       21:27           3       NaN  
3                       20:50           3       NaN  
4                       20:51           3       NaN  
...                       ...         ...       ...  
9067  491:1,492:5,493:1,494:2           3       NaN  
9068  496:1,497:3,468:1,495:7           3       NaN  
9069  497:2,498:1,468:2,495:8           3       NaN  
9070  496:1,497:3,468:1,495:7           3       NaN  
9071  497:2,498:1,468:2,495:8           3       NaN  

[9072 rows x 8 columns]

In [8]:
helsinki_nodes

stop_I        lat        lon             name
0          1  60.171270  24.956570       Kirkkokatu
1          2  60.172580  24.956554        Vironkatu
2          3  60.174180  24.956880   Meritullinkatu
3          4  60.174297  24.956514   Meritullinkatu
4          5  60.174280  24.960710  Liisanpuistikko
...      ...        ...        ...              ...
6981    7450  60.383837  25.277388      Kartanontie
6982    7522  60.370982  25.297348   Sommarnäsintie
6983    7523  60.370258  25.297900   Sommarnäsintie
6984    7524  60.367617  25.301282     Ollkärrintie
6985    7525  60.367260  25.301515     Ollkärrintie

[6986 rows x 4 columns]

In [10]:
# STEP 2
# convert to graph
G = convert_to_graph(helsinki_full)
pos = add_positions(G, helsinki_nodes)

# STEP 3
# convert walking network to seperate graph
G_walk = convert_to_graph(helsinki_walking)

In [ ]:
# make G connected by adding walking edges
make_walking_connected(G, G_walk)

In [ ]:
# STEP 4
results = percolation_experiments(G, 0, 1, 20, 1, 5, verbose=True)